Here is the Agent logic. Agent has two neural nets sharing the same input layer (observations). 
- The Actor NN encodes the policy and outputs the action to perform in the current state.
- The Critic NN encodes the value function and outputs the value of the current state.

The continuous action space case: In my case of a continuous action environnement (Va, elevator, aileron), the actor network returns the parameters of a Gaussian distribution (mean, std) per action, from which I will sample to get a discrete value to apply in the simulation.

Additional values like log probabilty and entropy are returned for future calculations.

In [ ]:
class Agent():
    def __init__(self):
        self.actor = ActorNeuralNetwork() # Arbitrary neural network
        self.critic = CriticNeuralNetwork() # Arbitrary neural network

    def get_value(self, obs):
        return self.critic(obs)
    
    def get_action_and_value(self, obs):
        action = self.actor(obs) # Sample action from actor, get Va, elevator, aileron
        value = self.critic(obs) # Get value of current obs
        logprob = self.actor.get_logprob()
        entropy = self.actor.get_entropy()
        return action, logprob, entropy, value


Here are the hyperparameters. Arbitrary values coming from the cleanRL implementation default parameters.
- `total_timesteps`: total timestep duration of the training
- `num_steps`: number of steps per policy rollout
- `batch_size` : total size of a policy rollout across num_envs parallel environements
- `num_updates`: number of updates of the neural nets
- `num_minibatches`: number of minibatches, a minibatch is a smaller batch on which every neural net update is going to be made
- `minibatch_size`: size of one minibatch: `batch_size // num_minibatches`
- `update_epochs`: number of updates per batch (rollout): one epoch contains `num_minibatches` neural net updates


In [ ]:
args = {
total_timesteps = 1500000,
num_steps = 2048,
num_envs = 2,
batch_size = num_envs * num_steps # 4096,
num_updates = total_timesteps // num_steps # 732,
num_minibatches = 32,
minibatch_size = batch_size // num_minibatches # 128,
update_epochs = 10,
gamma = 0.99,
gae_lambda = 0.95,
clip_coef = 0.2,
vf_coef = 0.5,
ent_coef = 0.0,
clip_vloss = True,
lr = 3e-4,
anneal_lr = True,
norm_adv = True,
}


Storage of various variables for containing the rollout relevant informations. Creating matrices initialized to zero with appropriate shape :
- `obs`: matrix containing all the obs for a rollout, `shape = (num_steps=2048, num_envs=2, num_channels=1, history_length=5, state_size=12)`, in my case I have an observation space composed of `num_channel = 1` (necessary for CNN layer), the history of my states `history_length = 5` and the state `[Va, roll, pitch, r, p, y, Va_err, roll_err, pitch_err, action_aileron_avg, action_elevator_avg, action_throttle_avg]`.
  - `Va`: airspeed
  - `r, p, y`: angular velocities
  - `action_<cmd>_avg`: average of 5 past actions for each of the 3 available commands.

- `actions`: matrix for all the performed actions in a rollout `shape = (num_steps=2048, num_envs=2, action_size=3)`
- `logprobs`: matrix for the all the log probabilities of the rollout
- `rewards`: matrix for the reward of the rollout
- `dones`: matrix for the done signals of the rollout (done is returned by the env when it reached a terminal state or it had to be truncated because of a timeout constraint or an observation out of bounds eg. diverging to NaN in my JSBSim bug case)
- `values`: matrix containing the values of the states reached during the rollout

In [ ]:
envs = Vector_of_Environments(num_envs) # Vector of parallelized environments
obs = matrix((num_steps, num_envs) + envs.single_env.observation_space.shape) # (2048, 2, 1, 5, 12)
actions = matrix((num_steps, num_envs) + envs.single_env.action_space.shape) # (2048, 2, 3)
logprobs = matrix((num_steps, num_envs)) # (2048, 2)
rewards = matrix((num_steps, num_envs)) # (2048, 2)
dones = matrix((num_steps, num_envs)) # (2048, 2)
values = matrix((num_steps, num_envs)) # (2048, 2)

- Create the agent object, optimizer with respect to the agent's parameters.  
- Start the game and by calling reset on the env in order to obtain the first observation.  
- Create a matrix holding the current done values while rolling out the policy.  
- Start the training loop by iterating until we reach the number of `num_updates` and if instructed, anneal the learning rate as the training loop advances --> Smaller updates the more training passes by.

In [ ]:
agent = Agent()
optimizer = Adam(agent.parameters(), lr=lr, eps=1e-5)
next_obs = envs.reset()
next_done = matrix(num_envs) # (2, 1)

for update in range(1, num_updates):
    if args.anneal_lr:
        frac = 1.0 - (update - 1.0) / num_updates
        lr_new = lr * frac
        optimizer.set_lr(lr_new)

Play the game, collect the rollout. Get the action to perform in the current observation and additional variables (values, logprobs). Log those single values in the rollout storage precedently created.

In [ ]:
    for step in range(0, args.num_steps):
        obs[step] = next_obs
        dones[step] = next_done

        action, logprob, _, value = agent.get_action_and_value(next_obs)
        actions[step] = action
        values[step] = value
        logprobs[step] = logprob

Step the environment with the selected action, get the next observation, the reward, two booleans indicating:
- if the env `terminated` because of a final state
- or if the env was `truncated` because of some time limit or abnormal behaviour  
Store the obtained reward in the appropriate rollout buffer.

In [ ]:
        next_obs, reward, terminated, truncated, infos = envs.step(action)
        next_done = terminated or truncated # error, should not confound terminated and truncated
        rewards[step] = reward
        # END OF ROLLOUT COLLECTION

Computation of the Advantage with the Generalized Advantage Estimator. [Schulman et al., 2018](https://arxiv.org/abs/1506.02438)  

For a bit of context on upcoming computations:
$V(s_t) = \mathbb{E}_{s_{t+1}:\infty} [\sum_{l=0}^{\infty} \gamma^l r_{t+l}]$ Is the Value function. It's the expected/average sum of rewards (discounted by a param $\gamma$, for caring more about immediate reward than far in time rewards) over all possible actions in the current state. It's a measure of the quality of a state.

$Q(s_t, a_t) = \mathbb{E}_{s_{t+1}:\infty, a_{t+1}:\infty} [\sum_{l=0}^{\infty} \gamma^l r_{t+l}]$ Is the Q-function, or the Action-Value function. It resembles the Value function, except that we are now measuring the expected sum of rewards but now for a specific action $a_t$ taken at time $t$ in state $s_t$. In other words it's a measure of the quality of taking action $a_t$ at state $s_t$.

$A(s_t, a_t) = Q(s_t, a_t) - V(s_t)$, is the advantage function, it's a difference between the action state pair quality and the overall quality of the current state. It gives a measure of how advantageous the action taken is relative to the other possible actions. If A<0, taking $a_t$ is worse than the average sum of rewards over all actions. If A>0, taking $a_t$ is better.  

Thanks to the TD error formula we know that: $Q(s_t, a_t) = r_t + \gamma V(s_{t+1})$. (since we have the entire rollout, we know what $r_t$ equals to).
This formula allows us:
- To obtain a less noisy estimate of the advantage. In fact, using the $Q(s_t, a_t) = \mathbb{E}_{s_{t+1}:\infty, a_{t+1}:\infty} [\sum_{l=0}^{\infty} \gamma^l r_{t+l}]$ would've been very dependent on the actions and transitions between states in a rollout due to the large sum of terms and having high variance between rollouts makes learning less stable and slower. Taking the estimation $V$ given by the critic neural net, gives bad values at the beginning of the training but it provides a steady approximation of Q throughout the training.

This TD error could also be generalized to all timesteps, eg. for 2 timesteps: $r_t + \gamma r_{t+1} + \gamma^2 V(s_{t+2})$.

The Advantage estimator at timestep $t$ for a single timestep lookahead becomes: $\hat{A}_t^{(1)} = \delta_t = r_t +\gamma V(s_{t+1}) - V(s)$.  
This can be generalized to k-steps lookahead eg. advantage $A_t^{(k)}$ for a k timestep look in the future: $A_t^{(k)} = \sum_{l=0}^{\infty} \gamma^l \delta_{t+l} = -V(s_t) + r_t + \gamma r_{t+1} + ... + \gamma^{k-1}r_{t+k-1} + \gamma^k V(s_{t+k})$

The Generalized Advantage Estimator GAE($\gamma, \lambda$) is defined as the exponentially weighted average of these k-step estimators see (16) for intermediate steps leading to this expression: $\hat{A_t^{GAE(\gamma, \lambda)} = \sum_{l=0}^{\infty}} = (\gamma \lambda)^l \delta_{t+l}$

Here is where the absence of distinction between termination or truncation of the current episode leads to unwanted behaviour.  
First, we get the value of the last observation of the rollout that is not comprised in `values[]` because this matrix fills at each action step, in the case of the last observation, no action step has been taken so we need to store this value by calling the `get_value()` method of the agent.  
We traverse the rollout from the last collected sample to the first one and we compute the advantage for each timestep of the rollout:
- If we are the last timestep of the rollout: set the boolean mask accordingly whether the last state of the rollout is also a final state of the env `1.0 - next_done=0 if state is done, =1 if state is not done`. Set the nextvalue variable to this "last_observation" value.
- Else, we set the boolean mask `is_nextnonterminal` for the entire rollout with the same logic. Set the nextvalue to the value at timestep `t+1`.
- Calculation of $\delta_t$ from the 1-step advantage estimator formula. The only difference occurs when the state is terminal, then `is_nextnonterminal = 0` and the "lookahead" term is nullified, because since we're in a terminal state there are no future rewards to obtain. (a difference should be made between termination and truncation here, in the truncated case, I would have future rewards to look to since the condition on interrupting the episode doesn't belong to the Markov Decision Process of my problem).
- The advantages are computed recursively from the end of the rollout following formula (16). At the end of the rollout we can only do a 1-step lookahead, the more we go back in time the more we compute k-step estimators based on the k-1-step estimator we obtained in the previous step loop.
- Finally the returns eg. the discouted sum of the rewards of the rollout can also be estimated by this lower variance estimator `advantages + values` rather that taking the high variance empirical form $G_t = \sum_{l=0}^{\infty} = \gamma^l r_{t+l}$

In [ ]:
    next_value = agent.get_value(next_obs) # here next_obs is the last obs of the rollout V(s_t+1)
    advantages = matrix(rewards.shape)
    last_adv = 0
    for t in reversed(range(args.num_steps)):
        if t == num_steps - 1: # if last step of the rollout
            is_nextnonterminal = 1.0 - next_done # boolean to indicate if next state is terminal (0: yes, 1: no)
            nextvalue = next_value
        else:
            is_nextnonterminal = 1.0 - dones[t+1]
            nextvalue = values[t+1]
        delta = rewards[t] + gamma * nextvalue * is_nextnonterminal - values[t]
        advantages[t] = last_adv = delta + args.gamma * args.gae_lambda * is_nextnonterminal * last_adv
    returns = advantages + values

    # flatten the num_envs dimension: instead of having (num_envs, num_steps, ...) we have (num_envs * num_steps, ...)
    b_obs = obs.reshape((-1,) + envs.single_observation_space.shape) # (4096, 1, 5, 12)
    b_logprobs = logprobs.reshape(-1) # (4096)
    b_actions = actions.reshape((-1,) + envs.single_action_space.shape) # (4096, 3)
    b_advantages = advantages.reshape(-1) # (4096)
    b_returns = returns.reshape(-1) # (4096)
    b_values = values.reshape(-1) # (4096)

### Optimizing the policy and value network. PPO Specific Logic.
Randomize the batch indices and split the batch in minibatches.
For a fixed amount of times, update the policy and the value function, proceed to traversing the batch, minibatch per minibatch, do forward pass, compute losses and gradients and backpropagate. There will be a total of `update_epochs * num_minibatches` updates per policy rollout.

#### Forward pass & Policy Loss
The probability ratio is defined as $r_t(\theta) = \frac{\pi_{\theta}(a_t | s_t)}{\pi_{\theta_{old}}(a_t | s_t)}$. The Advantage gives the information on whether the taken action is good or not compared to the rest of the actions of a given state. The idea of PPO is to limit updates that are to big eg. that move $r_t(\theta)$ away from 1.
Therefore the proposed PPO Policy objective is: $L^{(policy)}(\theta) = \mathbb{E}[\min(r_t(\theta)\hat{A}_t, \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon)\hat{A}_t)]$. Careful, here $r_t$ denotes the probabilty ratio between the old policy (used to produce the rollout and the new updated one) not the reward.
With this objective, we only ignore a big change in probability ratio if it makes our objective improve eg. $A>0$ and we include it when it makes the objective worse.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/recap.jpg)

Code:
- Compute the log probability ratio between the old policy (the one used for producing the rollouts, so take the logprob from the batch of rollout itself) and the newly updated one (obtained by forward passing the updated policy). Exponentiate this log probabililty ratio to get the ratio. Normally it's a quotient between those 2 probas, but since we're getting the logprob from the rollout and the forward pass we take the difference ($\log(\frac{a}{b}) = \log(a)-\log(b)$).
- Normalize advantages if required
- Piece the objective together (loss) turn it to a gradient descent by minimizing the loss instead of maximizing an objective: place minus signs for loss terms and take the max() instead of the min()


In [ ]:
    b_inds = np.arange(args.batch_size) # generating batch indices [0, 1, 2, ..., 4095]
    for epoch in range(args.update_epochs): # we update the policy for several epochs over the same batch/rollout
        random.shuffle(b_inds) # Stochastic gradient ascent : shuffle batch indices
        for start in range(0, args.batch_size, args.minibatch_size): # for each minibatch in the batch
            end = start + minibatch_size # move the minibatch window
            mb_inds = b_inds[start:end] # get the minibatch indices for this window

            # forward pass:
            _, newlogprob, entropy, newvalues = agent.get_action_and_value(b_obs[mb_inds], b_actions[mb_inds])
            logratio = newlogprob - b_logprobs[mb_inds] # logratio = log(pi(a|s)) - log(pi_old(a|s))
            ratio = logratio.exp() # take the exponential of the logratio to get the ratio

            mb_advantages = b_advantages[mb_inds]
            if args.norm_adv: # if asked to normalize advantages
                mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

            # policy loss
            pg_loss1 = -mb_advantages * ratio
            pg_loss2 = -mb_advantages * clip(ratio, 1.0 - clip_coef, 1.0 + clip_coef)
            pg_loss = max(pg_loss1, pg_loss2).mean()


#### Value loss and entropy loss $L_t^{VF}$
Normally value loss is a Mean Squared Error loss between the predicted values `newvalue` and the empirical return `b_returns` (sum of discounted rewards of the collected rollout). If enabled, there is also a clipping mechanism to promote conservative updates of the value function (critic) neural net:
- If the difference between `newvalues` obtained on last forward pass with the updated new value network and `b_values` obtained on the rollout collection are outside of certain bounds, we clip it.
- We add this clipped difference to the values obtained for the rollout which gives a series of clipped values `v_clipped`.
- This clipped values `v_clipped` serves now as the prediction of the value function / critic NN.
- We take the square of the error between this `v_clipped` prediction and the empirical returns `b_returns` to obtain the clipped loss `v_loss_clipped`.
- Just as in the policy loss, we take the max between the clipped loss `v_loss_clipped` and the unclipped traditional loss and take the mean to have the final loss.

cf. Image of Policy gradient loss, same logic replace $r_t(\theta)$ with the difference `newvalues - b_values`

Get the entropy loss $S[\pi_{theta}(s_t)]$ by getting the entropy mean of the action probability distribution of the actor neural net.

Piece the general objective function: $L_t^{\text{total}}(\theta) = L_t^{\text{policy}}(\theta) - c_1 L_t^{VF}(\theta) + c_2S[\pi_{\theta}](s_t)$

In [ ]:
            # Value loss
            newvalues = newvalues.view(-1) # flatten the values tensor (128, 1) to (128)
            if args.clip_vloss:
                v_loss_unclipped = (newvalues - b_returns[mb_inds]) ** 2
                v_clipped = b_values[mb_inds] + clip(
                    newvalues - b_values[mb_inds],
                    -args.clip_coef,
                    args.clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                v_loss_max = max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()
            else:
                v_loss = 0.5 * ((newvalues - b_returns[mb_inds]) ** 2).mean()

            entropy_loss = entropy.mean()
            loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef

Finally, compute gradients and propagate them to update the NNs params(clip the gradients in the meantime too)
Go back to the top and collect a new rollout etc...

In [ ]:
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
            optimizer.step()